# Chapter 05-03: 전이학습 — Feature Extraction & Fine-Tuning

## 학습 목표
- 전이학습(Transfer Learning)의 개념과 두 가지 전략을 이해한다
- EfficientNetB0로 Feature Extraction 파이프라인을 구현한다
- 새 분류 헤드를 추가하고 점진적으로 Fine-Tuning을 적용한다
- 단계별 학습률 설정의 중요성을 이해하고 적용한다

## 목차
1. [전이학습 전략 개요](#1)
2. [Feature Extraction 단계](#2)
3. [새 분류 헤드 추가](#3)
4. [Fine-Tuning 단계](#4)
5. [단계별 학습률 설정 가이드](#5)
6. [정리](#6)

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성 시드
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')

## 1. 전이학습 전략 개요 <a id='1'></a>

전이학습 전략

**Feature Extraction**: 사전 학습 모델을 고정(freeze)하고 새 분류 헤드만 학습
**Fine-Tuning**: 일부 레이어를 해동(unfreeze)하여 미세조정

데이터가 적으면 Feature Extraction, 많으면 Fine-Tuning이 유리

### 전략 선택 기준

| 상황 | 권장 전략 |
|------|----------|
| 데이터 적음 + 도메인 유사 | Feature Extraction |
| 데이터 적음 + 도메인 다름 | Feature Extraction + 조심스러운 Fine-Tuning |
| 데이터 많음 + 도메인 유사 | Fine-Tuning (전체 또는 상위 레이어) |
| 데이터 많음 + 도메인 다름 | 처음부터 학습 (또는 전체 Fine-Tuning) |

### 왜 낮은 학습률인가?
사전 학습 가중치는 이미 잘 최적화되어 있다. 높은 학습률로 Fine-Tuning하면 좋은 가중치를 파괴한다.
일반적으로 Feature Extraction 단계보다 **10~100배 낮은 학습률**을 사용한다.

## 2. Feature Extraction 단계 <a id='2'></a>

**기본 모델(Base Model)을 완전히 고정**하고, 우리가 추가하는 새 분류 헤드만 학습한다.

핵심 설정:
- `include_top=False`: ImageNet 분류 헤드 제거
- `base_model.trainable = False`: 기본 모델 가중치 동결
- `training=False`: BatchNormalization을 추론 모드로 실행 (중요!)

In [ ]:
# EfficientNetB0 기본 모델 로드 (ImageNet 사전 학습 가중치)
# include_top=False: 분류 헤드 제거, 특징 추출기만 사용
IMG_SIZE = 224
NUM_CLASSES = 5  # 예: Flowers 데이터셋 5개 클래스

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# 기본 모델 동결 (Feature Extraction 단계)
base_model.trainable = False

# 동결 확인
total_layers = len(base_model.layers)
trainable_layers = sum(1 for l in base_model.layers if l.trainable)
frozen_layers = total_layers - trainable_layers

print(f'EfficientNetB0 기본 모델 정보:')
print(f'  전체 레이어 수: {total_layers}')
print(f'  학습 가능 레이어: {trainable_layers}')
print(f'  동결된 레이어: {frozen_layers}')
print(f'  총 파라미터: {base_model.count_params():,}')
print(f'  학습 가능 파라미터: {sum([tf.size(w).numpy() for w in base_model.trainable_weights]):,}')
print()

# 기본 모델의 출력 형태 확인
dummy_input = tf.random.normal([1, IMG_SIZE, IMG_SIZE, 3])
base_output = base_model(dummy_input, training=False)
print(f'기본 모델 출력 형태: {base_output.shape}')
print(f'  → {base_output.shape[1]}×{base_output.shape[2]} 공간 그리드, {base_output.shape[3]} 채널')

## 3. 새 분류 헤드 추가 <a id='3'></a>

동결된 기본 모델 위에 작업별 분류 헤드를 추가한다.

권장 분류 헤드 구조:
```
GlobalAveragePooling2D → Dense(256, relu) → Dropout(0.3) → Dense(num_classes, softmax)
```

**왜 GlobalAveragePooling인가?**
- Flatten보다 파라미터 수가 훨씬 적다
- 공간 정보를 채널별로 압축하여 과적합 방지
- 입력 해상도에 무관하게 동작

In [ ]:
# 새 분류 헤드 추가 및 완전한 전이학습 모델 구성

def build_transfer_model(base_model, num_classes, dropout_rate=0.3):
    """
    전이학습 모델 구성 함수
    
    Args:
        base_model: 동결된 사전 학습 기본 모델
        num_classes: 분류 클래스 수
        dropout_rate: 드롭아웃 비율
    
    Returns:
        완성된 전이학습 모델
    """
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    # 기본 모델 통과 (training=False: BN을 추론 모드로 실행)
    # BN 레이어는 training=True이면 배치 통계를 업데이트하므로
    # 동결된 모델에서는 반드시 training=False로 설정
    x = base_model(inputs, training=False)
    
    # 분류 헤드
    # GlobalAveragePooling: (None, 7, 7, 1280) → (None, 1280)
    x = tf.keras.layers.GlobalAveragePooling2D(name='gap')(x)
    
    # 중간 Dense 레이어
    x = tf.keras.layers.Dense(256, activation='relu', name='dense_1')(x)
    
    # Dropout으로 과적합 방지
    x = tf.keras.layers.Dropout(dropout_rate, name='dropout')(x)
    
    # 최종 분류층
    outputs = tf.keras.layers.Dense(
        num_classes, activation='softmax', name='classifier'
    )(x)
    
    return tf.keras.Model(inputs, outputs, name='EfficientNetB0_Transfer')


# Feature Extraction 모델 생성
transfer_model = build_transfer_model(base_model, NUM_CLASSES, dropout_rate=0.3)
transfer_model.summary()

# 파라미터 요약
total_params     = transfer_model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in transfer_model.trainable_weights])
frozen_params    = total_params - trainable_params

print(f'\n파라미터 요약:')
print(f'  전체 파라미터:    {total_params:>10,}')
print(f'  학습 가능 파라미터: {trainable_params:>10,}  ← 새 분류 헤드만')
print(f'  동결된 파라미터:   {frozen_params:>10,}  ← EfficientNetB0 본체')
print(f'  학습 비율:         {trainable_params/total_params*100:.2f}%')

In [ ]:
# Feature Extraction 단계 학습 설정

# CIFAR-10을 예시로 사용 (실제로는 도메인 특화 데이터셋 사용)
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.squeeze()
y_test  = y_test.squeeze()

# 일부 클래스만 사용 (동물 5종: 비행기=0, 자동차=1, 새=2, 고양이=3, 사슴=4)
NUM_CLASSES = 5

# 클래스 0~4만 필터링
train_mask = y_train < NUM_CLASSES
test_mask  = y_test  < NUM_CLASSES

x_train_sub = x_train_raw[train_mask]
y_train_sub = y_train[train_mask]
x_test_sub  = x_test_raw[test_mask]
y_test_sub  = y_test[test_mask]

# 224x224로 리사이즈 + 정규화 (EfficientNetB0 입력 형식)
def preprocess(images, labels):
    # float32 변환 및 정규화
    images = tf.cast(images, tf.float32)
    # EfficientNet 전처리 (내장 전처리 사용 안할 경우 0-255 그대로 입력)
    images = tf.image.resize(images, [IMG_SIZE, IMG_SIZE])
    return images, labels

BATCH_SIZE = 32

train_ds = tf.data.Dataset.from_tensor_slices((x_train_sub, y_train_sub))
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_test_sub, y_test_sub))
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f'학습 샘플 수: {len(x_train_sub):,}')
print(f'검증 샘플 수: {len(x_test_sub):,}')

# Feature Extraction 단계 컴파일 (높은 학습률 가능 - 헤드만 학습)
transfer_model_fe = build_transfer_model(base_model, NUM_CLASSES)
transfer_model_fe.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # 헤드 학습 시 일반 학습률
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Feature Extraction 학습 (빠른 수렴)
history_fe = transfer_model_fe.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    verbose=1
)

print(f'\nFeature Extraction 최종 검증 정확도: {history_fe.history["val_accuracy"][-1]:.4f}')

## 4. Fine-Tuning 단계 <a id='4'></a>

Feature Extraction 학습 후, 기본 모델의 **상위 레이어를 해동**하여 전체 모델을 더 세밀하게 조정한다.

### Fine-Tuning 전략
1. Feature Extraction이 수렴할 때까지 먼저 학습
2. 기본 모델의 **하위 레이어는 유지** (저수준 특징 — 엣지, 텍스처)
3. **상위 레이어만 해동** (고수준 특징 — 형태, 의미론적 특징)
4. **매우 낮은 학습률** 사용 (1e-4 ~ 1e-5)

In [ ]:
# Fine-Tuning 단계 설정

# Feature Extraction으로 학습된 모델을 Fine-Tuning
fine_tune_model = transfer_model_fe

# 기본 모델 전체 해동
fine_tune_model.layers[1].trainable = True  # base_model은 두 번째 레이어

# 상위 레이어만 학습 가능하게 설정 (하위 레이어는 재동결)
# EfficientNetB0의 전체 레이어 중 마지막 30개만 해동
FINE_TUNE_AT = len(base_model.layers) - 30  # 상위 30개 레이어만 해동

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False  # 하위 레이어 재동결

for layer in base_model.layers[FINE_TUNE_AT:]:
    layer.trainable = True   # 상위 레이어 해동

# Fine-Tuning 후 파라미터 상태 확인
total_params     = fine_tune_model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in fine_tune_model.trainable_weights])
frozen_params    = total_params - trainable_params

print(f'Fine-Tuning 설정:')
print(f'  해동 시작 레이어: {FINE_TUNE_AT} (전체 {len(base_model.layers)}개 중)')
print(f'  학습 가능 파라미터: {trainable_params:>10,}')
print(f'  동결된 파라미터:   {frozen_params:>10,}')
print(f'  학습 비율:         {trainable_params/total_params*100:.2f}%')
print()

# Fine-Tuning 컴파일 (중요: Feature Extraction보다 훨씬 낮은 학습률)
# 학습률을 10배 낮게 설정하여 기존 가중치 보존
fine_tune_lr = 1e-4  # Feature Extraction의 1/10

fine_tune_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-Tuning 학습
history_ft = fine_tune_model.fit(
    train_ds,
    epochs=5,
    initial_epoch=5,        # Feature Extraction 이어서 학습
    validation_data=val_ds,
    verbose=1
)

print(f'\nFine-Tuning 최종 검증 정확도: {history_ft.history["val_accuracy"][-1]:.4f}')

In [ ]:
# Feature Extraction + Fine-Tuning 학습 곡선 통합 시각화

# 두 단계의 히스토리 합치기
acc     = history_fe.history['accuracy']     + history_ft.history['accuracy']
val_acc = history_fe.history['val_accuracy'] + history_ft.history['val_accuracy']
loss     = history_fe.history['loss']        + history_ft.history['loss']
val_loss = history_fe.history['val_loss']    + history_ft.history['val_loss']

epochs_fe = len(history_fe.history['accuracy'])
epochs_total = len(acc)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 정확도 그래프
ax1.plot(acc, label='학습 정확도', color='blue')
ax1.plot(val_acc, label='검증 정확도', color='orange')
ax1.axvline(x=epochs_fe - 0.5, color='red', linestyle='--', linewidth=2,
            label=f'Fine-Tuning 시작 (에폭 {epochs_fe})')
ax1.fill_betweenx([0, 1], 0, epochs_fe - 0.5, alpha=0.1, color='blue',
                   label='Feature Extraction 구간')
ax1.fill_betweenx([0, 1], epochs_fe - 0.5, epochs_total, alpha=0.1, color='green',
                   label='Fine-Tuning 구간')
ax1.set_xlabel('에폭')
ax1.set_ylabel('정확도')
ax1.set_title('학습/검증 정확도')
ax1.legend(loc='lower right', fontsize=8)
ax1.grid(True, alpha=0.3)

# 손실 그래프
ax2.plot(loss, label='학습 손실', color='blue')
ax2.plot(val_loss, label='검증 손실', color='orange')
ax2.axvline(x=epochs_fe - 0.5, color='red', linestyle='--', linewidth=2,
            label=f'Fine-Tuning 시작')
ax2.fill_betweenx([0, max(loss) * 1.1], 0, epochs_fe - 0.5,
                   alpha=0.1, color='blue')
ax2.fill_betweenx([0, max(loss) * 1.1], epochs_fe - 0.5, epochs_total,
                   alpha=0.1, color='green')
ax2.set_xlabel('에폭')
ax2.set_ylabel('손실')
ax2.set_title('학습/검증 손실')
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.suptitle('전이학습: Feature Extraction → Fine-Tuning', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 단계별 학습률 설정 가이드 <a id='5'></a>

### 학습률 선택 원칙

| 단계 | 학습률 범위 | 이유 |
|------|------------|------|
| Feature Extraction | $1 \times 10^{-3}$ ~ $1 \times 10^{-2}$ | 새 헤드만 학습, 자유롭게 설정 가능 |
| Fine-Tuning (상위 레이어) | $1 \times 10^{-5}$ ~ $1 \times 10^{-4}$ | 기존 가중치 보존, 조심스럽게 |
| 전체 Fine-Tuning | $1 \times 10^{-6}$ ~ $1 \times 10^{-5}$ | 매우 조심스럽게, 대용량 데이터 필요 |

In [ ]:
# 단계별 학습률 설정 가이드 코드

def get_transfer_learning_optimizer(stage='feature_extraction', base_lr=1e-3):
    """
    전이학습 단계에 맞는 최적화기 반환
    
    Args:
        stage: 학습 단계 ('feature_extraction' | 'fine_tuning' | 'full_fine_tuning')
        base_lr: Feature Extraction 기준 학습률
    
    Returns:
        Adam 최적화기
    """
    lr_scale = {
        'feature_extraction': 1.0,     # 기준 학습률 그대로
        'fine_tuning':        0.1,     # 1/10 수준
        'full_fine_tuning':   0.01,    # 1/100 수준
    }
    
    lr = base_lr * lr_scale.get(stage, 1.0)
    print(f'단계: {stage}, 학습률: {lr:.2e}')
    return tf.keras.optimizers.Adam(learning_rate=lr)


def apply_fine_tuning_schedule(model, base_model, fine_tune_at_percent=0.7):
    """
    모델의 상위 (1 - fine_tune_at_percent) 비율 레이어만 해동
    
    Args:
        model: 전이학습 모델
        base_model: 기본 모델 (model 내부의 레이어)
        fine_tune_at_percent: 동결 유지 비율 (0.7 = 하위 70% 동결)
    """
    total_layers = len(base_model.layers)
    freeze_until = int(total_layers * fine_tune_at_percent)
    
    # 기본 모델 전체 해동 후 선택적 재동결
    base_model.trainable = True
    
    for i, layer in enumerate(base_model.layers):
        if i < freeze_until:
            layer.trainable = False  # 하위 레이어 동결
    
    n_frozen   = sum(1 for l in base_model.layers if not l.trainable)
    n_trainable = total_layers - n_frozen
    
    print(f'Fine-Tuning 레이어 설정:')
    print(f'  전체: {total_layers}개')
    print(f'  동결: {n_frozen}개 (하위 {fine_tune_at_percent*100:.0f}%)')
    print(f'  학습: {n_trainable}개 (상위 {(1-fine_tune_at_percent)*100:.0f}%)')


# 단계별 학습률 확인
print('===== 전이학습 단계별 학습률 =====\n')
for stage in ['feature_extraction', 'fine_tuning', 'full_fine_tuning']:
    opt = get_transfer_learning_optimizer(stage, base_lr=1e-3)

print()

# Fine-Tuning 레이어 설정 예시
print('===== Fine-Tuning 레이어 설정 예시 =====\n')
temp_base = tf.keras.applications.EfficientNetB0(
    include_top=False, weights=None, input_shape=(224, 224, 3)
)
temp_model = build_transfer_model(temp_base, NUM_CLASSES)
apply_fine_tuning_schedule(temp_model, temp_base, fine_tune_at_percent=0.7)

# 학습률 스케줄러 예시
print('\n===== 학습률 스케줄러 예시 =====\n')

# Cosine Decay (Fine-Tuning 시 권장)
cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    alpha=1e-6  # 최솟값
)

# 학습률 시각화
steps = np.arange(0, 1000)
lrs = [cosine_decay(step).numpy() for step in steps]

plt.figure(figsize=(8, 4))
plt.plot(steps, lrs)
plt.xlabel('학습 스텝')
plt.ylabel('학습률')
plt.title('Cosine Decay 학습률 스케줄')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. 정리 <a id='6'></a>

### 전이학습 핵심 원칙

1. **Feature Extraction 먼저**: 항상 기본 모델을 동결한 채 분류 헤드를 먼저 학습시켜 수렴시킨다
2. **점진적 해동**: 상위 레이어부터 조금씩 해동하여 Fine-Tuning
3. **낮은 학습률**: Fine-Tuning 시 Feature Extraction의 1/10 ~ 1/100 학습률 사용
4. **BatchNorm 주의**: 동결된 모델의 BN은 `training=False`로 실행
5. **데이터 증강**: 적은 데이터에서 전이학습 시 데이터 증강 필수

### 학습률 요약
| 단계 | 권장 학습률 |
|------|------------|
| Feature Extraction | $10^{-3}$ |
| Fine-Tuning (상위) | $10^{-4}$ |
| Full Fine-Tuning | $10^{-5}$ |

### 다음 챕터 예고
**Chapter 05-04**: 객체 탐지(Object Detection) 입문 — 분류를 넘어 이미지에서 객체의 위치까지 찾아내는 기술을 학습한다.